In [6]:
import os
import sys
sys.path.append('Network/')
sys.path.append('NGEV/')
sys.path.append('Matrix/')

import makeTSnet as mts
import readNetwork as rn
import makeConstMat as mcm
import readSparseMat as rsm


In [2]:
# 実験するネットワークの名前を入れましょう
network_name = 'Sample'

In [3]:
# 必要なファイルがそろっているかを確認するセル(エラーが出たら必要なファイルを確認して用意すること)
root = os.path.dirname(os.path.abspath('.'))
root = os.path.join(root, '_sampleData', network_name)
print(root)
files = os.listdir(root)

files_is_ready = 0

if 'netname_net.tntp'.replace('netname', network_name) not in files:
    print('netname_net.tntp file is not exist.')
    files_is_ready = -1
if 'netname_node.tntp'.replace('netname', network_name) not in files:
    print('netname_node.tntp file is no exist.')
    files_is_ready = -1
if 'netname_trips.tntp'.replace('netname', network_name) not in files:
    print('netname_trips.tntp file is not exist.')
    files_is_ready = -1



scenario_set = [f for f in files if os.path.isdir(os.path.join(root, f))]
for scene in scenario_set:
    scene_files = os.listdir(os.path.join(root, scene))
    if 'netname_vu.tntp'.replace('netname', network_name) not in scene_files:
        print(scene + '/netname_vu.tntp file is not exist.')
        files_is_ready = -1


if files_is_ready == -1:
    print('FILE NOT READY ERROR!!!')

print(scenario_set)

c:\Users\tsuba\Documents\master_thesis\TNPandMS_experCode\_sampleData\Sample
['Scenario_0']


### ここから実験コード

In [4]:
# 必要データの生成
# capacity_scale: 道路容量を何倍にするか．時空間ネットワーク作成時に利用
# non_trip_cost: tripを実行しない場合のコスト．仮想ネットワーク作成時に利用
capacity_scale = 0.75
non_trip_cost = 20.0

In [8]:
for scenario in scenario_set:

    scenario_root = os.path.join(root, scenario)

    # 時空間ネットワークを作成
    original_links = rn.read_net(os.path.join(root, 'netname_net.tntp'.replace('netname', network_name)))
    num_zones = rn.read_num_zones(os.path.join(root, 'netname_net.tntp'.replace('netname', network_name)))
    ftn = rn.read_ftn(os.path.join(root, 'netname_net.tntp'.replace('netname', network_name)))
    original_nodes = rn.read_node(os.path.join(root, 'netname_node.tntp'.replace('netname', network_name)))
    num_times = rn.read_num_times(os.path.join(scenario_root, 'netname_vu.tntp'.replace('netname', network_name)))
    [TS_links, TS_nodes] = mts.make_TS_net(original_links, original_nodes, num_times, capacity_scale)

    # 時空間ネットワークを保存
    temp_root = os.path.join(scenario_root, 'TS_net')
    os.makedirs(temp_root, exist_ok=True)
    rn.write_net(os.path.join(temp_root, 'netname_ts_net.tntp'.replace('netname', network_name)), TS_links, num_zones*2, len(TS_nodes), ftn, len(TS_links))
    rn.write_node(os.path.join(temp_root, 'netname_ts_node.tntp'.replace('netname', network_name)), TS_nodes)

    # 時空間ネットワークに関するメモリを解放
    del TS_links, TS_nodes






    # 仮想ネットワークを作成
    [vehicle_info, user_info] = rn.read_vu(os.path.join(scenario_root, 'netname_vu.tntp'.replace('netname', network_name)))
    [vehicle_scost, user_scost] = rn.read_scost(os.path.join(scenario_root, 'netname_scost.tntp'.replace('netname', network_name)))
    virtual_root = os.path.join(scenario_root, 'virtual_net')
    const_root = os.path.join(scenario_root, 'constMat')

    # 車両側の仮想ネットワーク
    for vehicle_num in vehicle_info.keys():
        [VTS_links, VTS_nodes] = mts.make_vehicle_net(original_links, original_nodes, num_zones, num_times, vehicle_info[vehicle_num], vehicle_scost[vehicle_num], non_trip_cost)

        os.makedirs(os.path.join(virtual_root, 'vehicle', str(vehicle_num)), exist_ok=True)

        rn.write_net(os.path.join(virtual_root, 'vehicle', str(vehicle_num), 'netname_vir_net.tntp'.replace('netname', network_name)), VTS_links, num_zones*2, len(VTS_nodes), ftn, len(VTS_links))
        rn.write_node(os.path.join(virtual_root, 'vehicle', str(vehicle_num), 'netname_vir_node.tntp'.replace('netname', network_name)), VTS_nodes)

        # 係数行列を作成
        os.makedirs(os.path.join(const_root, 'vehicle', str(vehicle_num)), exist_ok=True)


    del VTS_links, VTS_nodes


    # 利用者側の仮想ネットワーク
    for user_num in user_info.keys():
        [UTS_links, UTS_nodes] = mts.make_user_net(original_links, original_nodes, num_zones, num_times, user_info[user_num], user_scost[user_num], non_trip_cost)
        
        os.makedirs(os.path.join(virtual_root, 'user', str(user_num)), exist_ok=True)

        rn.write_net(os.path.join(virtual_root, 'user', str(user_num), 'Sample_vir_net.tntp'), UTS_links, num_zones*2, len(UTS_nodes), 1, len(UTS_links))
        rn.write_node(os.path.join(virtual_root, 'user', str(user_num), 'Sample_vir_node.tntp'), UTS_nodes)

    del UTS_links, UTS_nodes


    print('finish making virtual_network !!!!!!\n\n\n')


start write net
end write net
start write nodes
end write nodes

start write net
end write net
start write nodes
end write nodes

start write net
end write net
start write nodes
end write nodes

finish making virtual_network !!!!!!



